## Adding a Key Value Pair to a Map

* put(key, value): used to add a key-value pair to a map
    - if the key is already present, then the old value will be replaced with the new one
    - if key is already present, put returns the existing value
        * if it's a new key, it will just return null
* putIfAbsent(key, value): also adds a key/value pair to the map only if the key is not already present and not associated to a null value
    - i.e. will replace a null value with the new value provided
    - useful for getting rid of null values in a map
* in the example below:
    - the code will fail with a NullPointerException b/c you cannot unbox a null Integer to an int value
    - map.values() is actually a Collection\<Integer\>
        * values() returns a Collection and Integer is the type of the values in the map
    - iterating on this collection produces instances of Integer and since we declared the variable value as int in the for-loop, the compiler will try to unbox the Integer returned by map.values()
        * this results in it failing and throwing a NullPointerException
    - to fix this, we can use the putIfAbsent() method to replace all null values with a default value, -1

In [1]:
Map<String, Integer> map = new HashMap<>();
map.put("one", 1);
map.put("two", null);
map.put("three", 3);
map.put("four", null);
map.put("five", 5);

// cannot unbox null Integer to an int value
for (int value: map.values()) {
    System.out.println("value = " + value);
}

EvalException: Cannot invoke "java.lang.Integer.intValue()" because the return value of "java.util.Iterator.next()" is null

In [3]:
Map<String, Integer> map = new HashMap<>();
map.put("one", 1);
map.put("two", null);
map.put("three", 3);
map.put("four", null);
map.put("five", 5);

// the putIfAbsent() method will replace all null values in the map
// to be a default value -1
for (String key: map.keySet()) {
    map.putIfAbsent(key, -1);
}

// will no longer throw a NullPointerException since all null values
// have been replaced with -1
for (int value: map.values()) {
    System.out.println("value = " + value);
}

value = -1
value = 1
value = -1
value = 3
value = 5


## Getting a Value From a Key

* get(key): returns a value bound to a given key
* getOrDefault(): can also return a value bound to a key but if the key is not in the map, the default value is returned instead

In [6]:
Map<Integer, String> map = new HashMap<>();

map.put(1, "one");
map.put(2, "two");
map.put(3, "three");

List<String> values = new ArrayList<>();
for (int i = 0; i < 5; i++) {
    values.add(map.getOrDefault(i, "UNDEFINED"));
}

System.out.println("values = " + values);

values = [UNDEFINED, one, two, three, UNDEFINED]


In [ ]:
// same result but done with a Stream

List<String> values = 
    IntStream.range(0, 5)
        .mapToObj(key -> map.getOrDefault(key, "UNDEFINED"))
        .collect(Collectors.toList());
System.out.println("values = " + values);

## Removing a Key from a Map

* remove(key): removes a key/value pair
    - also returns the value that was bound to that key so it may return null
* can be risky to remove a key/value pair blindly from a map if you don't know the value that is bound to that key
    - Java SE 8 added an overload that takes a value as a second argument
* remove(key, value): overloaded version of remove(key) that returns a boolean true if the key/value pair was removed from the map
    - the key/value pair will only be removed if the argument key and value match with a pair in the map
    - e.g. remove("three", 3) will be true if map contains "three": 3 and false if it contains "three": 4

## Checking for the Presence of a Key or a Value

* containsKey(key): returns a boolean if the map contains the key
* containsValue(value): returns a boolean if the map contains the value

## Checking for the Content of a Map

* isEmpty(): returns true if a map is empty
* size(): returns the number of key/value pairs in the map
* clear(): removes all contents of the map
* putAll(otherMap): adds all entries from otherMap into the current map
    - if there are duplicate keys, then otherMap's values will replace the current map's values

## Getting a View on the Keys, the Values or the Entries of a Map

* keySet(): returns an instance of Set containing the keys defined in the map
    - remember that keys in a map are unique so it makes sense to return a Set
* entrySet(): returns an instance of Set<Map.Entry> containing the key/value pairs contained in the map
    - same can be said of entries
    - you can only have unique entries since you must have unique keys so a Set makes sense
* values(): returns an instance of Collection containing the values present in the map
    - even though values are bounded by keys, there can be duplicate values in your map
    - therefore, it makes sense to house them in a Collection
* __these sets are _views_ backed by the current map so any changes made to the map is reflected in those views and vice versa__

In [10]:
Map<Integer, String> map = new HashMap<>();

map.put(1, "one");
map.put(2, "two");
map.put(3, "three");
map.put(4, "four");
map.put(5, "five");
map.put(6, "six");

Set<Integer> keys = map.keySet();
System.out.println("keys = " + keys);

Collection<String> values = map.values();
System.out.println("values = " + values);

Set<Map.Entry<Integer, String>> entries = map.entrySet();
System.out.println("entries = " + entries);

keys = [1, 2, 3, 4, 5, 6]
values = [one, two, three, four, five, six]
entries = [1=one, 2=two, 3=three, 4=four, 5=five, 6=six]


### Removing a Key From the Set of Keys

* modifying one of these sets will also be reflected in the map
* removing a key from the set returned by keySet() removes the corresponding key/value pair from the map

In [11]:
keys.remove(3);
entries.forEach(System.out::println);

1=one
2=two
4=four
5=five
6=six


### Removing a Value From the Collection of Values

* removing values from a collection is a bit more complicated since values are non-unique
* in this case, if there are duplicate values, the first occurrence of a key/value pair with the matching value will be removed
* in the example below:
    - the first key/value pair with a value of 3 is 3: "three"
    - thus, this is the one that is removed
* keep in mind, depending on the implementation, you cannot tell in advance what key/value pair will be found first
    - if it's a HashMap(), you cannot tell in advance
    - if it's a SortedMap, you can tell since the keys are sorted by their natural order or by some other comparator passed into it when it was initialized

In [13]:
Map<Integer, String> map = 
    Map.ofEntries(
        Map.entry(1, "one"),
        Map.entry(2, "two"),
        Map.entry(3, "three"),
        Map.entry(4, "four")
    );
map = new HashMap<>(map);
map.values().remove("three");
System.out.println("map = " + map);

map = {1=one, 2=two, 4=four}


*  you also don't have access to all operations on these map sets
    - e.g. you cannot add an element to the set of keys or to the collection of values
        * you'll get an UnsupportedOperationException
* also, if you need to iterate over the key/value pairs of a map, the best way to do that is to iterate over the set of key/value pairs directly
    - it is more efficient than iterating over the key set and retrieving the values from the map separately

In [14]:
// efficient way of getting the key/value pairs from a map
for (Map.Entry<Integer, String> entry : map.entrySet()) {
    System.out.println("entry = " + entry);
}

entry = 1=one
entry = 2=two
entry = 4=four
